# Cifar 10 - Darknet

In [1]:
%matplotlib inline

In [2]:
from fastai.conv_learner import *

## Dataset

In [3]:
PATH = Path("data/cifar10/")

In [4]:
os.makedirs(PATH, exist_ok=True)

In [5]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [6]:
stats = (np.array([ 0.4914 ,  0.48216,  0.44653]), np.array([ 0.24703,  0.24349,  0.26159]))

In [7]:
bs = 256
sz = 32

In [8]:
tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomFlip()], pad=sz//8)  # fastai uses reflection padding

In [9]:
data = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs)

## Architecture

**The conv layer does *not* need a bias because it is directly followed by a BatchNorm layer.**

In [10]:
def ConvLayer(n_in, n_out, kernel_size=3, stride=1):
    return nn.Sequential(
        nn.Conv2d(n_in, n_out, kernel_size=kernel_size, bias=False, stride=stride, padding=kernel_size//2),
        nn.BatchNorm2d(n_out, momentum=0.01),
        nn.LeakyReLU(negative_slope=0.1, inplace=True))

# the 'inplace' saves memory
# works with dropout too, activation functions and 
# arithmetic operations

**Bottleneck layer:**
The 'Wide Resnet' paper hints that something like this is better/faster becaue more computation in parallel:

```
self.conv1 = ConvLayer(n_in, n_in, kernel_size=1)
self.conv2 = ConvLayer(n_in, n_in, kernel_size=3)
```

In [11]:
class ResidualLayer(nn.Module):
    def __init__(self, n_in):  # n_in == n_out
        super().__init__()
        self.conv1 = ConvLayer(n_in, n_in//2, kernel_size=1)  # basically n//2 weighted averages
        self.conv2 = ConvLayer(n_in//2, n_in, kernel_size=3)
        
    def forward(self, x):
        return x.add(self.conv2(self.conv1(x)))

`make_group_layer` does the following:
 
 1. The ConvLayer doubles the number of filters (and if stride=2 reduces the hight and width by 2)
 1. The ConvLayer is followed by `n_blocks` residual layers which don't change the dimensionality

In [12]:
class DarkNet(nn.Module):
    def make_group_layer(self, ch_in, n_blocks, stride=1):
        return [ConvLayer(ch_in, ch_in * 2, stride=stride)] + [(ResidualLayer(ch_in * 2)) for i in range(n_blocks)]
    
    def __init__(self, n_blocks, n_classes, nf = 32):
        super().__init__()
        layers = [ConvLayer(3, nf, kernel_size=3, stride=1)]
        
        for i, nb in enumerate(n_blocks):
            layers += self.make_group_layer(nf, nb, stride=2 - (i==1))  # cifar10 imgs are small, can't half grid size every layer
            nf *= 2
        layers += [nn.AdaptiveAvgPool2d(1), Flatten(), nn.Linear(nf, n_classes)]
        
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.layers(x)

**Important notice: When using `nn.AdaptiveAvgPool` instead of `nn.AvgPool` you are *not* tied to a specific input image size for any fully convolutional architecture like resnets or VGG!**

**Also the nice way utilized here to create a model: Make a list of layers and then use `nn.Sequential(*layers)`.**

In [13]:
model = DarkNet([1, 2, 4, 6, 3], n_classes=10, nf=32) 

In [14]:
model = nn.DataParallel(model, device_ids=None)  # if you have several GPUs use device_ids=[1, 2, 3]

In [15]:
lr = 1.3

In [16]:
learn = ConvLearner.from_model_data(model, data)

In [17]:
learn.crit = nn.CrossEntropyLoss()

In [18]:
learn.metrics = [accuracy]

In [19]:
wd = 1e-4

In [20]:
%time learn.fit(lr, 1, wds=wd, cycle_len=30, use_clr_beta=(20, 20, 0.95, 0.85))

epoch      trn_loss   val_loss   accuracy                   
    0      2.181148   2.967308   0.1161    
    1      1.793812   1.927621   0.2715                     
    2      1.560586   1.824395   0.3395                     
    3      1.404661   1.678231   0.4158                     
    4      1.218635   1.307029   0.5204                     
    5      1.075651   1.404319   0.512                      
    6      0.943258   1.132617   0.5939                      
    7      0.858313   1.017639   0.638                       
    8      0.772648   1.209958   0.5769                      
    9      0.730512   2.093357   0.4441                      
    10     0.686704   1.054698   0.6551                      
    11     0.6303     0.978388   0.6768                      
    12     0.614091   0.8449     0.7186                      
    13     0.581461   0.713726   0.757                       
    14     0.546164   0.729345   0.7531                      
    15     0.520433   0.942237  

[0.2667015944480896, 0.9129]

**The model in the notebook `Resnet from scratch - CIFAR 10.ipynb` reached only an accuracy of 0.8369%!**